In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from math import ceil, sqrt
import copy

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
class MultiDimensionalBS:
    def __init__(self, initial_prices, interest_rate, dividends, volatilities, correlation_matrix):
        self.initial_prices = initial_prices
        self.interest_rate = interest_rate
        self.dividends = dividends
        self.volatilities = volatilities
        self.correlation_matrix = correlation_matrix
        self.dimension = len(initial_prices)
        self.var_cov = np.linalg.cholesky(np.matmul(np.diag(self.volatilities), np.matmul(self.correlation_matrix, np.diag(self.volatilities))))
    
    def Simulate(self, expiry, nb_time_discr, number_paths):
                
        diffusion_list = np.linspace(0, expiry, nb_time_discr + 1)
        last_index = len(diffusion_list) - 1
        delta_time = expiry / last_index
        dW = np.sqrt(delta_time) * np.random.randn(number_paths, last_index, self.dimension)
        dW = np.append(np.zeros((number_paths, 1, self.dimension)), dW, axis=1)
        cst = self.interest_rate - self.dividends - 0.5 * (self.volatilities ** 2)
        W = np.cumsum(np.matmul(dW, self.var_cov), axis = 1)
        S = self.initial_prices * np.exp(cst * diffusion_list[np.newaxis, :, np.newaxis] + W)
                    
        return S
    
    def Simulate_Next_State(self, current_price, current_time, diffusion_times, nb_paths):
        delta_time = diffusion_times[0] - current_time
        dW = np.sqrt(delta_time) * np.random.randn(nb_paths, len(diffusion_times), self.dimension)
        cst = self.interest_rate - self.dividends - 0.5 * (self.volatilities ** 2)
        W = np.cumsum(np.matmul(dW, self.var_cov), axis = 1)
        S = current_price[:, np.newaxis, np.newaxis, :] * np.exp(cst * (diffusion_times[np.newaxis, :, np.newaxis] - current_time) + W)
                    
        return S

In [3]:
class Payoff:
    def __init__(self, strike, option_type):
        self.strike = strike
        self.option_type = option_type
        
    def value(self, asset_prices, interest_rate, dates, k=0):
        if (self.option_type == 0):
            return np.exp(-interest_rate * dates[np.newaxis, k:]) * np.maximum(asset_prices.max(axis = 2) - self.strike, 0)
        
        elif (self.option_type == 1):
            return np.exp(-interest_rate * dates[np.newaxis, k:]) * np.maximum(self.strike - asset_prices.min(axis = 2), 0)
        
        elif (self.option_type == 2):
            d = len(asset_prices[0][0])
            np.exp(-interest_rate * dates[np.newaxis, k:]) * np.maximum(asset_prices.prod(axis = 2)**(1/d) - self.strike, 0)

In [4]:
##### BS parameters

nb_assets = 2
interest_rate = 0.05
T = 3
N = 9
strike = 100
my_payoff = Payoff(strike, 0)

initial_prices = np.full(nb_assets, 100)
dividends = np.full(nb_assets, 0.1)
volatilities = np.full(nb_assets, 0.2)
diffusion_times = np.linspace(0, T, N + 1)
correlation_matrix = np.eye(nb_assets)

# # Train Neural Net

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
def model_train(nb_epochs, inputs_size, batch_size, nb_batches, learning_rate, N):

    
    nn_model = nn.Sequential(nn.Linear(nb_assets + 1, q1), nn.ReLU(), nn.BatchNorm1d(q1), nn.Linear(q1, q2), 
                                        nn.ReLU(), nn.BatchNorm1d(q1), torch.nn.Linear(q2, 1), torch.nn.Sigmoid())
    
    f = [copy.deepcopy(nn_model).to(device) for i in range(N - 1)]
    optimizers = [torch.optim.Adam(f[i].parameters(), lr = learning_rate) for i in range(N - 1)]
    BS = MultiDimensionalBS(initial_prices, interest_rate, dividends, volatilities, correlation_matrix)

    for epoch in range(nb_epochs):
        
        S_train = BS.Simulate(T, N, inputs_size)
        x = torch.from_numpy(S_train).float().to(device)
        y = torch.unsqueeze(torch.from_numpy(my_payoff.value(S_train, interest_rate, diffusion_times)), dim=2).float().to(device)
        
        # condition finale

        payoff_tau = [y[i * batch_size : min((i + 1) * batch_size, inputs_size) , N] for i in range(nb_batches)]
        
        # bakward induction
        
        for n in range(N - 1, 0, -1):
            
            inputs = torch.cat((x[:,n], y[:,n]), 1)
            payoffs = [y[i * batch_size : min((i + 1) * batch_size, inputs_size) , n] for i in range(nb_batches)]
            
            for batch in range(nb_batches):
                this_batch = inputs[batch * batch_size : min((batch + 1) * batch_size, inputs_size), :].to(device)
                optimizers[n - 1].zero_grad()
                F_n = f[n - 1](this_batch)
                f_n = (F_n >= 0.5).detach().float()
                payoffs_batch = y[batch * batch_size : min((batch + 1) * batch_size, inputs_size) , n]
                payoff_tau[batch] = payoffs_batch * f_n + payoff_tau[batch] * (1 - f_n)
                z = torch.mean(torch.mul(torch.subtract(payoff_tau[batch], payoffs_batch), F_n))
                z.backward()
                optimizers[n - 1].step()                            
    return {'model' : f}

In [7]:
# NN parameters

nb_epochs = 200 + nb_assets
batch_size = 512
inputs_size = 8192
q1 = q2 = 40 + nb_assets
learning_rate = 0.001
nb_batches = ceil(inputs_size / batch_size)

In [ ]:
training = model_train(nb_epochs, inputs_size, batch_size, nb_batches, learning_rate, N)
#training['rewards']

# # Lower Bound

In [10]:
K_L = 4096000
K_L = 2000000
BS_lower = MultiDimensionalBS(initial_prices, interest_rate, dividends, volatilities, correlation_matrix)
S_lower = BS_lower.Simulate(T, N, K_L)

x = torch.from_numpy(S_lower).float()
y = torch.unsqueeze(torch.from_numpy(my_payoff.value(S_lower, interest_rate, diffusion_times)), dim=2).float()

In [11]:
# initialization

tau = y[:,N].new_full(y[:,N].shape, N).to(device)
payoff_tau = y[:,N].to(device)

# backward induction

for n in range(N - 1, 0, -1):
    inputs = torch.cat((x[:,n], y[:,n]), 1)
    f_n = (training['model'][n - 1](inputs.to(device)) >= 0.5).detach().float()
    tau = n * f_n + tau * (1 - f_n)
    payoffs = y[:, n].to(device)
    payoff_tau = payoffs * f_n + payoff_tau * (1 - f_n)
    
    print(n, 'reward :', payoff_tau.mean())
    
# initial date

payoffs = y[:,0].to(device)
f_n = (payoffs >= (payoff_tau).mean()).detach().float()
tau *= (1 - f_n)
payoff_tau = payoffs * f_n + payoff_tau * (1 - f_n)

lower = payoff_tau
lower_bound = lower.mean().item()

print(0, 'reward :', payoff_tau.mean())

8 reward : tensor(11.8431, device='cuda:0')
7 reward : tensor(12.3798, device='cuda:0')
6 reward : tensor(12.8413, device='cuda:0')
5 reward : tensor(13.2336, device='cuda:0')
4 reward : tensor(13.5401, device='cuda:0')
3 reward : tensor(13.7519, device='cuda:0')
2 reward : tensor(13.8619, device='cuda:0')
1 reward : tensor(13.8788, device='cuda:0')
0 reward : tensor(13.8788, device='cuda:0')


In [11]:
print('tau :', round(tau.mean().item(), 3), 'lower bound :', round(lower_bound, 3))

tau : 7.137 lower bound : 13.871


# # Upper Bound

In [12]:
J = 16384
K_U = 1024

BS_upper = MultiDimensionalBS(initial_prices, interest_rate, dividends, volatilities, correlation_matrix)

In [13]:
S_upper = BS_upper.Simulate(T, N, K_U)
x = torch.from_numpy(S_upper).float()
y = torch.unsqueeze(torch.from_numpy(my_payoff.value(S_upper, interest_rate, diffusion_times)), dim=2).float()

In [ ]:
S_cond = BS_upper.Simulate_Next_State(S_upper[:, N - 1, :], diffusion_times[N - 1], diffusion_times[N:], J)
y_cond = [my_payoff.value(S_cond[k], interest_rate, diffusion_times[N:]) for k in range(K_U)]
C_n_1 = torch.from_numpy(np.mean(np.squeeze([y_cond[k] for k in range(K_U)], axis = 2), axis = 1)).to(device)
mg_increments = torch.from_numpy(np.array([[0.0] * K_U] * (N + 1)))
mg_increments[N] = y[:, N, 0].to(device) - C_n_1
f_n = [[torch.from_numpy(np.array([[1.0] * 1] * J)).to(device)] * 1] * K_U

for n in range(N - 1, 0, -1):
    S_cond = BS_upper.Simulate_Next_State(S_upper[:, n - 1, :], diffusion_times[n - 1], diffusion_times[n:], J)
    y_cond = [torch.unsqueeze(torch.from_numpy(my_payoff.value(S_cond[k], interest_rate, diffusion_times[n:])), dim = 2).float() for k in range(K_U)]
    S_cond = torch.from_numpy(S_cond).float()
    
    inputs = [[torch.cat((S_cond[k][:, m], y_cond[k][:, m]), 1) for m in range(0, N - n)] for k in range(K_U)]
    payoff_tau = [y_cond[k][:, N - n].to(device) for k in range(K_U)]
        
    if (not (n == N - 1)):
        f_n = [[(training['model'][n + m - 1](inputs[k][m].to(device)) >= 0.5).detach().float() for m in range(0, N - n)] for k in range(K_U)]
        
        for m in range(N - n - 1, -1, -1):
            payoff_tau = [y_cond[k][:, m, :].to(device) * f_n[k][m] + (1 - f_n[k][m]) * payoff_tau[k] for k in range(K_U)]
    
    C_n = C_n_1
    C_n_1 = [torch.mean(torch.squeeze(payoff_tau[k], dim = 1)).item() for k in range(K_U)]
    C_n_1 = torch.from_numpy(np.array(C_n_1)).to(device)
        
    inputs = torch.cat((x[:,n], y[:,n]), 1)
    f_n = (training['model'][n - 1](inputs.to(device)) >= 0.5).detach().float()
    mg_increments[n] = (y[:, n, 0].to(device) - C_n) * torch.squeeze(f_n, dim = 1) + C_n - C_n_1
    
mg_doob = torch.cumsum(torch.transpose(mg_increments, 0, 1), dim = 1)

upper = (y[:,:,0].to(device) - mg_doob.to(device)).max(dim = 1).values
upper_bound = upper.mean().item()

In [ ]:
print('upper bound :', round(upper_bound, 3))

In [ ]:
IC_inf = round(lower_bound - 1.96 * torch.std(lower).item() / sqrt(K_L), 3)
IC_sup = round(upper_bound + 1.96 * torch.std(upper).item() / sqrt(K_U), 3)

print('- lower bound :', round(lower_bound, 3))
print('- upper bound :', round(upper_bound, 3))
print('- point estimator :', round((lower_bound + upper_bound) / 2.0, 3))
print('')
print('IC 90% :', (IC_inf, IC_sup))

In [ ]:
# to do : qq tests et graphes

##### 